# 📄 DocumentSegmenterAgent - Classificador de Partes do Ofício

Este notebook define o agente responsável por **classificar e segmentar** as partes de um ofício judicial de quebra de sigilo bancário.

## Objetivo
Antes de extrair subsídios, investigados ou datas, precisamos **entender a estrutura do documento** e **classificar cada trecho** para:
1. Identificar quais partes são relevantes para o Banco X
2. Descartar ruído (headers OCR, fundamentação jurídica extensa, etc.)
3. Direcionar cada trecho para o agente especializado correto

---

## 1. Contexto do Sistema SIMBA

### O que é o SIMBA?
O SIMBA é um sistema do **Banco X** para processamento automatizado de ofícios judiciais de quebra de sigilo bancário. Ele recebe documentos digitalizados (OCR) de varas criminais, promotorias e outros órgãos, e precisa:

1. **Identificar** quem são os investigados (pessoas físicas/jurídicas)
2. **Extrair** quais informações bancárias estão sendo solicitadas (subsídios)
3. **Determinar** os períodos de cada solicitação
4. **Validar** se o pedido é realmente direcionado ao Banco X
5. **Gerar** a resposta estruturada para atendimento

### O que é Quebra de Sigilo Bancário?
É uma determinação judicial que obriga instituições financeiras a fornecerem informações protegidas pelo sigilo bancário sobre clientes específicos. Pode incluir:
- Extratos de contas
- Movimentações financeiras
- Contratos
- Dados cadastrais
- Histórico de transações

### O que é um Subsídio?
No contexto do SIMBA, **subsídio** é cada tipo de informação/documento bancário que pode ser solicitado em um ofício. Exemplos:
- Extrato de conta corrente
- Fatura de cartão de crédito
- Contratos de empréstimo
- Logs de acesso digital
- Consulta ao CCS/SISBACEN

## 2. Estrutura Típica de um Ofício Judicial

Um ofício de quebra de sigilo pode conter **qualquer combinação** dos seguintes elementos, **sem padrão fixo**:

| Categoria | Descrição | Relevância para Extração |
|-----------|-----------|-------------------------|
| `header_ocr` | Artefatos do scanner: "Scanned by...", "Página 1/5", linhas "___" | ❌ Ignorar |
| `header_juridico` | Cabeçalho institucional: "PODER JUDICIÁRIO", "COMARCA DE..." | ⚪ Contexto |
| `identificacao_documento` | Número do processo, data, natureza do procedimento | ⚪ Metadados |
| `descricao_caso` | Contexto da investigação, descrição do crime | ⚪ Contexto |
| `fundamentacao_juridica` | Artigos de lei, justificativas, precedentes | ❌ Ignorar |
| `solicitacao_quebra_sigilo` | **O pedido em si**: produtos, serviços, períodos | ✅ **CRÍTICO** |
| `informacoes_investigado` | Nomes, CPFs, CNPJs, endereços | ✅ **CRÍTICO** |
| `identificacao_instituicao` | Para qual banco/instituição é o pedido | ✅ **CRÍTICO** |
| `prazo` | Prazo para cumprimento | ⚪ Metadados |
| `assinatura` | Assinatura do juiz/promotor | ⚪ Metadados |
| `ruido` | Texto ilegível, caracteres corrompidos | ❌ Ignorar |

## 3. Imports e Configuração

In [ ]:
import json
from typing import List, Optional, Literal
from enum import Enum
from pydantic import BaseModel, Field
from datetime import datetime

## 4. Definição dos Schemas (Response Format)

In [ ]:
# ============================================================
# ENUMS E TIPOS
# ============================================================

class CategoriaConteudo(str, Enum):
    """Categorias possíveis para classificação de trechos do ofício."""
    HEADER_OCR = "header_ocr"
    HEADER_JURIDICO = "header_juridico"
    IDENTIFICACAO_DOCUMENTO = "identificacao_documento"
    DESCRICAO_CASO = "descricao_caso"
    FUNDAMENTACAO_JURIDICA = "fundamentacao_juridica"
    SOLICITACAO_QUEBRA_SIGILO = "solicitacao_quebra_sigilo"
    INFORMACOES_INVESTIGADO = "informacoes_investigado"
    IDENTIFICACAO_INSTITUICAO = "identificacao_instituicao"
    PRAZO = "prazo"
    ASSINATURA = "assinatura"
    RUIDO = "ruido"


class TipoDestinatario(str, Enum):
    """Tipos de instituição destinatária."""
    BANCO_X = "banco_x"
    INSTITUICAO_FINANCEIRA_GENERICA = "instituicao_financeira_generica"
    BACEN = "bacen"
    RECEITA_FEDERAL = "receita_federal"
    OPERADORA_TELEFONIA = "operadora_telefonia"
    OUTRO_BANCO = "outro_banco"
    OUTRA_INSTITUICAO = "outra_instituicao"
    NAO_ESPECIFICADO = "nao_especificado"


class NivelRelevancia(str, Enum):
    """Nível de relevância do trecho para o Banco X."""
    CRITICO = "critico"      # Deve ser processado (solicitações, investigados)
    CONTEXTO = "contexto"    # Útil para entendimento, mas não essencial
    IGNORAR = "ignorar"      # Pode ser descartado

In [ ]:
# ============================================================
# SCHEMAS DE RESPOSTA (Pydantic)
# ============================================================

class SegmentoClassificado(BaseModel):
    """Representa um segmento/trecho do texto classificado."""
    
    segment_id: int = Field(
        description="ID sequencial do segmento (1, 2, 3...)"
    )
    
    texto: str = Field(
        description="Texto completo do segmento"
    )
    
    categoria: Literal[
        "header_ocr", "header_juridico", "identificacao_documento",
        "descricao_caso", "fundamentacao_juridica", "solicitacao_quebra_sigilo",
        "informacoes_investigado", "identificacao_instituicao", "prazo",
        "assinatura", "ruido"
    ] = Field(
        description="Categoria do conteúdo deste segmento"
    )
    
    destinatario: Literal[
        "banco_x", "instituicao_financeira_generica", "bacen",
        "receita_federal", "operadora_telefonia", "outro_banco",
        "outra_instituicao", "nao_especificado"
    ] = Field(
        description="Instituição destinatária identificada neste segmento (se aplicável)"
    )
    
    relevancia_banco_x: Literal["critico", "contexto", "ignorar"] = Field(
        description="Nível de relevância deste segmento para o Banco X"
    )
    
    contem_investigado: bool = Field(
        description="Se o segmento contém informações de investigado (nome, CPF, CNPJ)"
    )
    
    contem_subsidio: bool = Field(
        description="Se o segmento contém solicitação de subsídios/documentos"
    )
    
    contem_periodo: bool = Field(
        description="Se o segmento contém datas ou períodos"
    )
    
    justificativa_classificacao: str = Field(
        description="Explicação do porquê esta classificação foi atribuída"
    )
    
    palavras_chave_encontradas: List[str] = Field(
        default=[],
        description="Palavras-chave que ajudaram na classificação"
    )


class MetadadosDocumento(BaseModel):
    """Metadados extraídos do documento."""
    
    numero_processo: Optional[str] = Field(
        default=None,
        description="Número do processo judicial, se identificado"
    )
    
    numero_oficio: Optional[str] = Field(
        default=None,
        description="Número do ofício, se identificado"
    )
    
    data_documento: Optional[str] = Field(
        default=None,
        description="Data do documento, se identificada"
    )
    
    orgao_emissor: Optional[str] = Field(
        default=None,
        description="Órgão que emitiu o ofício (vara, promotoria, etc.)"
    )
    
    autoridade_signataria: Optional[str] = Field(
        default=None,
        description="Nome do juiz/promotor que assina"
    )
    
    prazo_cumprimento: Optional[str] = Field(
        default=None,
        description="Prazo para cumprimento da ordem"
    )


class QualidadeOCR(BaseModel):
    """Avaliação da qualidade do OCR."""
    
    score: float = Field(
        description="Score de qualidade de 0.0 a 1.0"
    )
    
    classificacao: Literal["boa", "media", "ruim"] = Field(
        description="Classificação qualitativa"
    )
    
    problemas_detectados: List[str] = Field(
        default=[],
        description="Lista de problemas identificados no OCR"
    )
    
    recomendacao: Optional[str] = Field(
        default=None,
        description="Recomendação sobre o processamento"
    )


class ResultadoSegmentacao(BaseModel):
    """Resultado completo da segmentação do documento."""
    
    timestamp: str = Field(
        default_factory=lambda: datetime.now().isoformat(),
        description="Timestamp da análise"
    )
    
    qualidade_ocr: QualidadeOCR = Field(
        description="Avaliação da qualidade do OCR"
    )
    
    metadados: MetadadosDocumento = Field(
        description="Metadados extraídos do documento"
    )
    
    total_segmentos: int = Field(
        description="Número total de segmentos identificados"
    )
    
    segmentos: List[SegmentoClassificado] = Field(
        description="Lista de todos os segmentos classificados"
    )
    
    resumo_relevancia: dict = Field(
        description="Contagem de segmentos por nível de relevância"
    )
    
    segmentos_criticos_banco_x: List[int] = Field(
        description="IDs dos segmentos críticos para o Banco X"
    )
    
    destinatarios_identificados: List[str] = Field(
        description="Lista de destinatários únicos identificados"
    )
    
    documento_relevante_banco_x: bool = Field(
        description="Se o documento como um todo é relevante para o Banco X"
    )
    
    justificativa_relevancia: str = Field(
        description="Explicação sobre a relevância do documento para o Banco X"
    )

In [ ]:
# ============================================================
# JSON SCHEMA PARA OPENAI (response_format)
# ============================================================

RESPONSE_FORMAT_JSON_SCHEMA = {
    "type": "json_schema",
    "json_schema": {
        "name": "resultado_segmentacao",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "qualidade_ocr": {
                    "type": "object",
                    "properties": {
                        "score": {"type": "number", "minimum": 0, "maximum": 1},
                        "classificacao": {"type": "string", "enum": ["boa", "media", "ruim"]},
                        "problemas_detectados": {"type": "array", "items": {"type": "string"}},
                        "recomendacao": {"type": ["string", "null"]}
                    },
                    "required": ["score", "classificacao", "problemas_detectados"],
                    "additionalProperties": False
                },
                "metadados": {
                    "type": "object",
                    "properties": {
                        "numero_processo": {"type": ["string", "null"]},
                        "numero_oficio": {"type": ["string", "null"]},
                        "data_documento": {"type": ["string", "null"]},
                        "orgao_emissor": {"type": ["string", "null"]},
                        "autoridade_signataria": {"type": ["string", "null"]},
                        "prazo_cumprimento": {"type": ["string", "null"]}
                    },
                    "required": [],
                    "additionalProperties": False
                },
                "total_segmentos": {"type": "integer"},
                "segmentos": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "segment_id": {"type": "integer"},
                            "texto": {"type": "string"},
                            "categoria": {
                                "type": "string",
                                "enum": [
                                    "header_ocr", "header_juridico", "identificacao_documento",
                                    "descricao_caso", "fundamentacao_juridica", "solicitacao_quebra_sigilo",
                                    "informacoes_investigado", "identificacao_instituicao", "prazo",
                                    "assinatura", "ruido"
                                ]
                            },
                            "destinatario": {
                                "type": "string",
                                "enum": [
                                    "banco_x", "instituicao_financeira_generica", "bacen",
                                    "receita_federal", "operadora_telefonia", "outro_banco",
                                    "outra_instituicao", "nao_especificado"
                                ]
                            },
                            "relevancia_banco_x": {
                                "type": "string",
                                "enum": ["critico", "contexto", "ignorar"]
                            },
                            "contem_investigado": {"type": "boolean"},
                            "contem_subsidio": {"type": "boolean"},
                            "contem_periodo": {"type": "boolean"},
                            "justificativa_classificacao": {"type": "string"},
                            "palavras_chave_encontradas": {
                                "type": "array",
                                "items": {"type": "string"}
                            }
                        },
                        "required": [
                            "segment_id", "texto", "categoria", "destinatario",
                            "relevancia_banco_x", "contem_investigado", "contem_subsidio",
                            "contem_periodo", "justificativa_classificacao", "palavras_chave_encontradas"
                        ],
                        "additionalProperties": False
                    }
                },
                "resumo_relevancia": {
                    "type": "object",
                    "properties": {
                        "critico": {"type": "integer"},
                        "contexto": {"type": "integer"},
                        "ignorar": {"type": "integer"}
                    },
                    "required": ["critico", "contexto", "ignorar"],
                    "additionalProperties": False
                },
                "segmentos_criticos_banco_x": {
                    "type": "array",
                    "items": {"type": "integer"}
                },
                "destinatarios_identificados": {
                    "type": "array",
                    "items": {"type": "string"}
                },
                "documento_relevante_banco_x": {"type": "boolean"},
                "justificativa_relevancia": {"type": "string"}
            },
            "required": [
                "qualidade_ocr", "metadados", "total_segmentos", "segmentos",
                "resumo_relevancia", "segmentos_criticos_banco_x", "destinatarios_identificados",
                "documento_relevante_banco_x", "justificativa_relevancia"
            ],
            "additionalProperties": False
        }
    }
}

print("✅ JSON Schema definido para response_format")

## 5. System Prompt do DocumentSegmenterAgent

In [ ]:
SYSTEM_PROMPT_SEGMENTER = """
# AGENTE DE SEGMENTAÇÃO E CLASSIFICAÇÃO DE OFÍCIOS JUDICIAIS

Você é um agente especializado do sistema SIMBA do Banco X, responsável por analisar e 
classificar as diferentes partes de ofícios judiciais de quebra de sigilo bancário.

---

## CONTEXTO DO SISTEMA SIMBA

### O que é o SIMBA?
O SIMBA (Sistema de Informações para Mandados Bancários) é a plataforma do Banco X para 
processamento automatizado de ofícios judiciais. Ele recebe documentos digitalizados via OCR 
e precisa extrair informações estruturadas para atendimento das ordens judiciais.

### O que é Quebra de Sigilo Bancário?
É uma determinação judicial (art. 4º da LC 105/2001) que obriga instituições financeiras a 
fornecerem informações protegidas sobre clientes específicos para fins de investigação.

### O que é um Subsídio?
No contexto SIMBA, "subsídio" é cada tipo de informação/documento bancário solicitado:
- Extratos (conta corrente, poupança, investimentos)
- Faturas de cartão de crédito
- Contratos (empréstimos, financiamentos)
- Dados cadastrais
- Transferências (DOC, TED, PIX)
- Logs de acesso digital
- Consultas ao CCS/SISBACEN
- Entre outros

---

## SUA TAREFA

Você deve:
1. **Segmentar** o texto em parágrafos/blocos lógicos
2. **Classificar** cada segmento em uma categoria
3. **Identificar** o destinatário de cada trecho (quando aplicável)
4. **Determinar** a relevância para o Banco X
5. **Extrair** metadados do documento
6. **Avaliar** a qualidade do OCR

---

## CATEGORIAS DE CLASSIFICAÇÃO

### 1. header_ocr
**O que é:** Artefatos gerados pelo scanner/OCR, não fazem parte do documento original.
**Como identificar:**
- "Scanned by CamScanner", "Digitized by..."
- "Página X de Y", "Page X/Y"
- Linhas de underscores: "_______________"
- Códigos de barras textualizados
- Timestamps de digitalização
**Relevância:** IGNORAR

### 2. header_juridico
**O que é:** Cabeçalho institucional do documento jurídico.
**Como identificar:**
- "PODER JUDICIÁRIO", "MINISTÉRIO PÚBLICO"
- "TRIBUNAL DE JUSTIÇA DO ESTADO DE..."
- "COMARCA DE...", "FORO...", "VARA..."
- Brasões, logotipos (descritos)
**Relevância:** CONTEXTO

### 3. identificacao_documento
**O que é:** Dados que identificam o documento específico.
**Como identificar:**
- "Processo nº", "Autos nº", "Procedimento nº"
- "OFÍCIO Nº", "MANDADO Nº"
- "Natureza:", "Classe:"
- Data do documento
- "Ref:", "Assunto:"
**Relevância:** CONTEXTO (metadados úteis)

### 4. descricao_caso
**O que é:** Narrativa sobre a investigação/processo.
**Como identificar:**
- "Trata-se de investigação..."
- "Nos autos do inquérito que apura..."
- Descrição de crimes investigados
- Histórico do processo
- "O investigado é acusado de..."
**Relevância:** CONTEXTO

### 5. fundamentacao_juridica
**O que é:** Justificativas legais para a quebra de sigilo.
**Como identificar:**
- Citações de artigos: "Art. 4º da LC 105/2001"
- "Conforme jurisprudência..."
- "Nos termos do art..."
- Referências a decisões anteriores
- Longas justificativas legais
**Relevância:** IGNORAR (não necessário para extração)

### 6. solicitacao_quebra_sigilo ⭐ CRÍTICO
**O que é:** O PEDIDO EM SI - quais informações estão sendo solicitadas.
**Como identificar:**
- "DETERMINO a quebra de sigilo..."
- "Solicito o envio de..."
- "Requisito as seguintes informações..."
- "Forneça:", "Providencie:"
- Listas de documentos/extratos/informações
- "1. Extratos de conta corrente; 2. Faturas..."
- Menção a tipos de documentos bancários
**Relevância:** CRÍTICO ✅

### 7. informacoes_investigado ⭐ CRÍTICO
**O que é:** Dados dos alvos da quebra de sigilo.
**Como identificar:**
- Nomes próprios em destaque/maiúsculas
- CPF: XXX.XXX.XXX-XX
- CNPJ: XX.XXX.XXX/XXXX-XX
- "Investigado:", "Réu:", "Indiciado:"
- RG, endereços, filiação
- "do(a) cliente", "do(a) correntista"
**Relevância:** CRÍTICO ✅

### 8. identificacao_instituicao ⭐ CRÍTICO
**O que é:** Identificação de PARA QUEM é o pedido.
**Como identificar:**
- "Ao Banco X", "À instituição financeira"
- "OFICIE-SE ao...", "Expeça-se ofício ao..."
- "Senhor Gerente do Banco..."
- Endereços de agências bancárias
- "Às instituições financeiras em geral"
**Relevância:** CRÍTICO ✅

### 9. prazo
**O que é:** Prazo para cumprimento da ordem.
**Como identificar:**
- "Prazo: X dias"
- "No prazo de..."
- "Em até X dias úteis"
- "Sob pena de..."
**Relevância:** CONTEXTO

### 10. assinatura
**O que é:** Assinatura e identificação do signatário.
**Como identificar:**
- "Dr./Dra. [Nome]"
- "Juiz de Direito", "Promotor de Justiça"
- "Delegado de Polícia"
- Assinaturas digitais, certificados
- Local e data ao final
**Relevância:** CONTEXTO

### 11. ruido
**O que é:** Texto ilegível ou corrompido pelo OCR.
**Como identificar:**
- Muitos caracteres especiais sem sentido: "@#$%&*"
- Palavras quebradas incompreensíveis
- Sequências aleatórias de letras
- Texto que não forma frases coerentes
**Relevância:** IGNORAR

---

## REGRAS DE IDENTIFICAÇÃO DE DESTINATÁRIO

### banco_x
Quando menciona EXPLICITAMENTE:
- "Banco X", "BANCO X S/A", "Banco X S.A."
- Variações do nome do Banco X

### instituicao_financeira_generica
Quando menciona de forma GENÉRICA:
- "instituição financeira", "instituições bancárias"
- "bancos em geral", "estabelecimentos bancários"
- "todas as instituições financeiras"
→ **CONSIDERAR RELEVANTE para Banco X**

### bacen
Quando é EXCLUSIVAMENTE para o Banco Central:
- "Oficie-se ao BACEN", "Ao Banco Central do Brasil"
- Pedido de informações do CCS enviado ao BCB
→ **NÃO relevante** (a menos que peça ao Banco X para consultar)

⚠️ **ATENÇÃO:** Se o texto pede que o BANCO X consulte o CCS/SISBACEN, 
então É RELEVANTE para o Banco X!

### receita_federal
- "Receita Federal", "RFB", "Fisco"
- Quebra de sigilo fiscal
→ **NÃO relevante**

### operadora_telefonia
- Vivo, Claro, TIM, Oi, Nextel
- Sigilo telefônico/telemático
→ **NÃO relevante**

### outro_banco
Quando menciona OUTRO banco específico:
- Itaú, Bradesco, Santander, Caixa, etc.
- Bancos que NÃO fazem parte do grupo Banco X
→ **NÃO relevante** (se exclusivo para outro banco)

### outra_instituicao
- Seguradoras, corretoras, cartórios
- Órgãos públicos diversos
→ **NÃO relevante**

### nao_especificado
Quando NÃO menciona nenhuma instituição:
- Pedido genérico sem destinatário explícito
→ **PRESUMIR relevante** para instituição financeira (inclui Banco X)

---

## REGRAS DE RELEVÂNCIA PARA O BANCO X

### CRÍTICO (deve ser processado)
- Segmentos com `categoria` = solicitacao_quebra_sigilo, informacoes_investigado, identificacao_instituicao
- E `destinatario` = banco_x, instituicao_financeira_generica, ou nao_especificado

### CONTEXTO (útil mas não essencial)
- header_juridico, identificacao_documento, descricao_caso, prazo, assinatura
- Ou segmentos críticos para outras instituições (manter para referência)

### IGNORAR (descartar)
- header_ocr, fundamentacao_juridica (muito longa), ruido
- Segmentos exclusivos para instituições não relacionadas ao Banco X

---

## AVALIAÇÃO DE QUALIDADE DO OCR

Calcule um score de 0.0 a 1.0 baseado em:

| Indicador | Impacto no Score |
|-----------|------------------|
| Muitos caracteres especiais isolados (@#$%&) | -0.2 |
| Palavras quebradas por hífen incorreto | -0.1 |
| Espaçamento irregular | -0.1 |
| Linhas com apenas símbolos | -0.1 |
| Texto coerente e legível | +0.3 |
| Estrutura de parágrafos clara | +0.2 |
| Números e datas bem formatados | +0.1 |

Classificação:
- **boa**: score >= 0.7
- **media**: 0.4 <= score < 0.7
- **ruim**: score < 0.4

---

## INSTRUÇÕES DE SEGMENTAÇÃO

1. **Divida o texto em parágrafos/blocos lógicos**
   - Use quebras de linha duplas como separadores principais
   - Mantenha listas numeradas juntas
   - Não quebre frases no meio

2. **Para cada segmento, determine:**
   - Qual a categoria mais apropriada
   - Se há um destinatário identificável
   - Se contém dados de investigado (nome/CPF/CNPJ)
   - Se contém solicitação de subsídios
   - Se contém datas/períodos

3. **Seja conservador:**
   - Na dúvida entre duas categorias, escolha a mais específica
   - Se um segmento tem múltiplos propósitos, classifique pelo principal
   - Não invente informações que não estão no texto

4. **Extraia metadados quando encontrar:**
   - Número do processo/ofício
   - Data do documento
   - Órgão emissor
   - Autoridade signatária
   - Prazo de cumprimento

---

## EXEMPLOS DE CLASSIFICAÇÃO

### Exemplo 1: header_ocr
```
Scanned by CamScanner
_______________________________________________
```
→ categoria: header_ocr | relevancia: ignorar

### Exemplo 2: solicitacao_quebra_sigilo + identificacao_instituicao
```
OFICIE-SE AO BANCO X S/A para que forneça, no prazo de 15 dias:
1. Extratos de conta corrente do período de jan/2023 a dez/2024;
2. Faturas de cartão de crédito;
3. Contratos de empréstimo vigentes.
```
→ categoria: solicitacao_quebra_sigilo | destinatario: banco_x | relevancia: critico

### Exemplo 3: informacoes_investigado
```
Investigado: JOÃO DA SILVA SANTOS
CPF: 123.456.789-00
RG: 12.345.678-9 SSP/SP
```
→ categoria: informacoes_investigado | contem_investigado: true | relevancia: critico

### Exemplo 4: fundamentacao_juridica (IGNORAR)
```
Considerando o disposto no art. 4º, §1º, da Lei Complementar nº 105/2001, 
que autoriza a quebra de sigilo bancário mediante requisição judicial, e 
tendo em vista a jurisprudência consolidada do STF no sentido de que...
[texto longo de justificativa legal]
```
→ categoria: fundamentacao_juridica | relevancia: ignorar

### Exemplo 5: Múltiplos destinatários
```
I - OFICIE-SE AO BACEN para informações do CCS;
II - OFICIE-SE ÀS INSTITUIÇÕES FINANCEIRAS para extratos;
III - OFICIE-SE À RECEITA FEDERAL para declarações de IR.
```
→ Segmentar em 3 partes:
   - I: destinatario: bacen, relevancia: contexto
   - II: destinatario: instituicao_financeira_generica, relevancia: critico
   - III: destinatario: receita_federal, relevancia: ignorar

---

## FORMATO DE SAÍDA

Retorne um JSON seguindo EXATAMENTE o schema definido.
Seja preciso, não alucine, e justifique cada classificação.
"""

print(f"✅ System Prompt definido ({len(SYSTEM_PROMPT_SEGMENTER)} caracteres)")

## 6. User Prompt Template

In [ ]:
USER_PROMPT_TEMPLATE = """
Analise e classifique o seguinte texto de ofício judicial de quebra de sigilo bancário.

## TEXTO DO OFÍCIO:

```
{texto_oficio}
```

## INSTRUÇÕES:

1. Avalie a qualidade do OCR (score de 0 a 1)
2. Extraia os metadados disponíveis (número do processo, data, etc.)
3. Segmente o texto em blocos lógicos
4. Classifique cada segmento conforme as categorias definidas
5. Identifique o destinatário de cada segmento (quando aplicável)
6. Determine a relevância para o Banco X
7. Marque quais segmentos contêm investigados, subsídios ou períodos

## LEMBRE-SE:

- Segmentos para "instituição financeira genérica" ou "não especificado" SÃO relevantes para o Banco X
- Segmentos EXCLUSIVOS para BACEN, Receita, outros bancos NÃO são relevantes
- Se pedir ao Banco X para consultar CCS/SISBACEN, É relevante
- Extraia o texto completo de cada segmento, não resuma

Retorne o JSON conforme o schema definido.
"""

def get_user_prompt(texto_oficio: str) -> str:
    return USER_PROMPT_TEMPLATE.format(texto_oficio=texto_oficio)

print("✅ User Prompt Template definido")

## 7. Tool de Segmentação (smolagents)

In [ ]:
from smolagents import Tool

class DocumentSegmenterTool(Tool):
    """
    Tool para segmentar e classificar partes de um ofício judicial.
    Identifica categorias, destinatários e relevância para o Banco X.
    """
    
    name = "document_segmenter"
    description = """Segmenta e classifica as partes de um ofício judicial de quebra de sigilo bancário.
    Identifica:
    - Categoria de cada trecho (header, solicitação, investigado, etc.)
    - Destinatário (Banco X, BACEN, Receita, etc.)
    - Relevância para o Banco X (crítico, contexto, ignorar)
    - Se contém dados de investigados, subsídios ou períodos
    Retorna JSON estruturado com a segmentação completa."""
    
    inputs = {
        "texto_oficio": {
            "type": "string",
            "description": "Texto completo do ofício judicial (pode ser OCR)"
        }
    }
    output_type = "string"
    
    def __init__(self, model_name: str = "gpt-4o", **kwargs):
        super().__init__(**kwargs)
        self.model_name = model_name
        self._client = None
    
    @property
    def client(self):
        if self._client is None:
            from openai import OpenAI
            self._client = OpenAI()
        return self._client
    
    def forward(self, texto_oficio: str) -> str:
        """
        Executa a segmentação e classificação do ofício.
        
        Args:
            texto_oficio: Texto do ofício judicial
        
        Returns:
            JSON string com resultado da segmentação
        """
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT_SEGMENTER},
                    {"role": "user", "content": get_user_prompt(texto_oficio)}
                ],
                temperature=0.1,
                response_format=RESPONSE_FORMAT_JSON_SCHEMA
            )
            
            resultado = json.loads(response.choices[0].message.content)
            resultado["timestamp"] = datetime.now().isoformat()
            resultado["modelo_utilizado"] = self.model_name
            
            return json.dumps(resultado, ensure_ascii=False, indent=2)
            
        except Exception as e:
            return json.dumps({
                "erro": str(e),
                "timestamp": datetime.now().isoformat()
            }, ensure_ascii=False)


print("✅ DocumentSegmenterTool definida")

## 8. Tools Auxiliares (Heurísticas)

In [ ]:
import re

class HeuristicAnalyzerTool(Tool):
    """
    Tool para análise heurística (regex/keywords) do texto.
    Executa ANTES do LLM para pré-identificar padrões.
    """
    
    name = "heuristic_analyzer"
    description = """Analisa o texto usando heurísticas (regex e palavras-chave) para 
    pré-identificar padrões como CPFs, CNPJs, datas, instituições e expressões jurídicas.
    Útil para validação cruzada com o resultado do LLM."""
    
    inputs = {
        "texto": {
            "type": "string",
            "description": "Texto a ser analisado"
        }
    }
    output_type = "string"
    
    # Padrões regex
    PATTERNS = {
        "cpf": r"\b\d{3}[.-]?\d{3}[.-]?\d{3}[.-]?\d{2}\b",
        "cnpj": r"\b\d{2}[.-]?\d{3}[.-]?\d{3}[/]?\d{4}[.-]?\d{2}\b",
        "processo": r"\b\d{7}[.-]?\d{2}[.-]?\d{4}[.-]?\d[.-]?\d{2}[.-]?\d{4}\b",
        "data_completa": r"\b\d{1,2}[/.-]\d{1,2}[/.-]\d{2,4}\b",
        "periodo": r"(?:período|de|entre)\s+(?:\d{1,2}[/.-]\d{1,2}[/.-]\d{2,4})\s+(?:a|até|e)\s+(?:\d{1,2}[/.-]\d{1,2}[/.-]\d{2,4})",
        "valor_monetario": r"R\$\s*[\d.,]+",
    }
    
    # Keywords por categoria
    KEYWORDS = {
        "instituicao_banco_x": ["banco x", "banco x s/a", "banco x s.a."],
        "instituicao_generica": ["instituição financeira", "instituições bancárias", "bancos em geral", "estabelecimentos bancários"],
        "bacen": ["bacen", "banco central", "bcb", "sisbacen", "ccs"],
        "receita_federal": ["receita federal", "rfb", "fisco", "sigilo fiscal"],
        "operadora": ["vivo", "claro", "tim", "oi", "nextel", "sigilo telefônico", "sigilo telemático"],
        "outros_bancos": ["itaú", "bradesco", "santander", "caixa econômica", "banco do brasil", "nubank", "inter"],
        "expressoes_quebra": ["quebra de sigilo", "determino a quebra", "oficie-se", "requisito", "solicito", "forneça"],
        "subsidios": ["extrato", "fatura", "cartão", "conta corrente", "poupança", "investimento", "empréstimo", 
                      "financiamento", "pix", "ted", "doc", "cheque", "cadastr", "contrato"],
        "header_ocr": ["scanned", "digitized", "página", "page", "___"],
    }
    
    def forward(self, texto: str) -> str:
        texto_lower = texto.lower()
        resultado = {
            "padroes_encontrados": {},
            "keywords_encontradas": {},
            "metricas_qualidade": {},
            "sugestoes_categoria": []
        }
        
        # Buscar padrões regex
        for nome, pattern in self.PATTERNS.items():
            matches = re.findall(pattern, texto, re.IGNORECASE)
            if matches:
                resultado["padroes_encontrados"][nome] = list(set(matches))
        
        # Buscar keywords
        for categoria, keywords in self.KEYWORDS.items():
            encontradas = [kw for kw in keywords if kw in texto_lower]
            if encontradas:
                resultado["keywords_encontradas"][categoria] = encontradas
        
        # Métricas de qualidade
        resultado["metricas_qualidade"] = {
            "total_caracteres": len(texto),
            "total_palavras": len(texto.split()),
            "proporcao_especiais": len(re.findall(r"[^\w\s.,;:!?()-]", texto)) / max(len(texto), 1),
            "linhas_vazias": texto.count("\n\n"),
            "tem_cpf": "cpf" in resultado["padroes_encontrados"],
            "tem_cnpj": "cnpj" in resultado["padroes_encontrados"],
            "tem_processo": "processo" in resultado["padroes_encontrados"],
        }
        
        # Sugestões de categoria baseadas em keywords
        if "expressoes_quebra" in resultado["keywords_encontradas"]:
            resultado["sugestoes_categoria"].append("solicitacao_quebra_sigilo")
        if resultado["metricas_qualidade"]["tem_cpf"] or resultado["metricas_qualidade"]["tem_cnpj"]:
            resultado["sugestoes_categoria"].append("informacoes_investigado")
        if "instituicao_banco_x" in resultado["keywords_encontradas"]:
            resultado["sugestoes_categoria"].append("identificacao_instituicao")
        if "header_ocr" in resultado["keywords_encontradas"]:
            resultado["sugestoes_categoria"].append("header_ocr")
        
        return json.dumps(resultado, ensure_ascii=False, indent=2)


print("✅ HeuristicAnalyzerTool definida")

In [ ]:
class OCRQualityTool(Tool):
    """
    Tool para avaliar a qualidade do OCR de forma determinística.
    """
    
    name = "ocr_quality_checker"
    description = """Avalia a qualidade do texto OCR usando métricas determinísticas.
    Retorna score de 0 a 1 e lista de problemas detectados."""
    
    inputs = {
        "texto": {
            "type": "string",
            "description": "Texto OCR a ser avaliado"
        }
    }
    output_type = "string"
    
    def forward(self, texto: str) -> str:
        problemas = []
        score = 1.0
        
        # 1. Proporção de caracteres especiais estranhos
        chars_estranhos = len(re.findall(r"[^\w\s.,;:!?()\-/\"'@#$%&*+=\[\]{}|\\<>]", texto))
        proporcao_estranhos = chars_estranhos / max(len(texto), 1)
        if proporcao_estranhos > 0.05:
            score -= 0.2
            problemas.append(f"Alta proporção de caracteres estranhos ({proporcao_estranhos:.2%})")
        
        # 2. Palavras quebradas por hífen no meio
        palavras_quebradas = len(re.findall(r"\w+-\s*\n\s*\w+", texto))
        if palavras_quebradas > 5:
            score -= 0.15
            problemas.append(f"Muitas palavras quebradas por hífen ({palavras_quebradas})")
        
        # 3. Linhas com apenas símbolos
        linhas = texto.split("\n")
        linhas_simbolos = sum(1 for l in linhas if l.strip() and not re.search(r"[a-zA-Z0-9]", l))
        if linhas_simbolos > 3:
            score -= 0.1
            problemas.append(f"Linhas com apenas símbolos ({linhas_simbolos})")
        
        # 4. Espaçamento irregular
        espacos_multiplos = len(re.findall(r"  +", texto))
        if espacos_multiplos > 20:
            score -= 0.1
            problemas.append(f"Espaçamento irregular ({espacos_multiplos} ocorrências)")
        
        # 5. Sequências de letras sem sentido (3+ consoantes seguidas)
        sem_sentido = len(re.findall(r"[bcdfghjklmnpqrstvwxyz]{4,}", texto.lower()))
        if sem_sentido > 10:
            score -= 0.15
            problemas.append(f"Possíveis palavras corrompidas ({sem_sentido})")
        
        # Garantir score entre 0 e 1
        score = max(0.0, min(1.0, score))
        
        # Classificação
        if score >= 0.7:
            classificacao = "boa"
            recomendacao = "Processar normalmente"
        elif score >= 0.4:
            classificacao = "media"
            recomendacao = "Processar com atenção extra na validação"
        else:
            classificacao = "ruim"
            recomendacao = "Considerar revisão humana antes do processamento"
        
        resultado = {
            "score": round(score, 2),
            "classificacao": classificacao,
            "problemas_detectados": problemas,
            "recomendacao": recomendacao,
            "metricas_detalhadas": {
                "total_caracteres": len(texto),
                "proporcao_caracteres_estranhos": round(proporcao_estranhos, 4),
                "palavras_quebradas": palavras_quebradas,
                "linhas_apenas_simbolos": linhas_simbolos,
                "espacos_multiplos": espacos_multiplos
            }
        }
        
        return json.dumps(resultado, ensure_ascii=False, indent=2)


print("✅ OCRQualityTool definida")

## 9. Exemplo de Uso Completo

In [ ]:
# Texto de exemplo para teste
TEXTO_EXEMPLO = """
Scanned by CamScanner
_______________________________________________

PODER JUDICIÁRIO DO ESTADO DE SÃO PAULO
COMARCA DE SÃO PAULO - FORO CENTRAL CRIMINAL
1ª VARA DE INQUÉRITOS POLICIAIS

Processo nº 0001234-56.2024.8.26.0050
Natureza: Inquérito Policial
Assunto: Quebra de Sigilo Bancário

OFÍCIO Nº 1234/2024

Ao Ilustríssimo Senhor Gerente do BANCO X S/A
Departamento Jurídico
Av. das Finanças, 1000 - São Paulo/SP

Senhor Gerente,

Nos autos do inquérito policial em epígrafe, que investiga crimes de lavagem de dinheiro 
e organização criminosa previstos na Lei 9.613/98, DETERMINO a quebra de sigilo bancário 
dos investigados abaixo qualificados:

1) JOÃO DA SILVA SANTOS
   CPF: 123.456.789-00
   RG: 12.345.678-9 SSP/SP

2) EMPRESA COMERCIAL ABC LTDA
   CNPJ: 12.345.678/0001-90

Considerando o disposto no art. 4º, §1º, da Lei Complementar nº 105/2001, que autoriza 
a quebra de sigilo bancário mediante requisição judicial, e tendo em vista os elementos 
colhidos na investigação que demonstram a necessidade da medida para apuração dos fatos.

Solicito que Vossa Senhoria providencie o envio das seguintes informações e documentos, 
relativos ao período de 01/01/2022 a 31/12/2024:

1. Extratos completos de todas as contas correntes e poupanças;
2. Faturas detalhadas de cartão de crédito;
3. Cópias de todos os cheques emitidos e compensados;
4. Relação de todas as transferências PIX, DOC e TED realizadas e recebidas;
5. Contratos de empréstimos e financiamentos;
6. Dados cadastrais completos;
7. Logs de acesso ao internet banking e aplicativo móvel.

OFICIE-SE também ao BANCO CENTRAL DO BRASIL para que forneça informações do CCS 
sobre relacionamentos financeiros dos investigados.

OFICIE-SE ainda à RECEITA FEDERAL para envio das declarações de IR dos últimos 5 anos.

Prazo para cumprimento: 15 (quinze) dias úteis, sob pena de responsabilização.

São Paulo, 15 de novembro de 2024.

Dr. Fulano de Tal
Juiz de Direito
"""

print("✅ Texto de exemplo carregado")
print(f"   Tamanho: {len(TEXTO_EXEMPLO)} caracteres")

In [ ]:
# Testar tools heurísticas
print("🔍 TESTE 1: Análise Heurística")
print("="*60)

heuristic_tool = HeuristicAnalyzerTool()
resultado_heuristica = heuristic_tool.forward(TEXTO_EXEMPLO)
print(resultado_heuristica)

In [ ]:
# Testar qualidade OCR
print("\n🔍 TESTE 2: Qualidade do OCR")
print("="*60)

ocr_tool = OCRQualityTool()
resultado_ocr = ocr_tool.forward(TEXTO_EXEMPLO)
print(resultado_ocr)

In [ ]:
# Testar segmentação completa (requer API key)
print("\n🔍 TESTE 3: Segmentação Completa com LLM")
print("="*60)

# Descomente para executar (requer OPENAI_API_KEY)
# segmenter_tool = DocumentSegmenterTool(model_name="gpt-4o")
# resultado_segmentacao = segmenter_tool.forward(TEXTO_EXEMPLO)
# print(resultado_segmentacao)

## 10. Resumo da Arquitetura

### Componentes Definidos:

| Componente | Tipo | Função |
|------------|------|--------|
| `SYSTEM_PROMPT_SEGMENTER` | Prompt | Instrui o LLM sobre contexto SIMBA, categorias, regras |
| `RESPONSE_FORMAT_JSON_SCHEMA` | Schema | Define estrutura exata da saída JSON |
| `DocumentSegmenterTool` | Tool (smolagents) | Executa segmentação via LLM |
| `HeuristicAnalyzerTool` | Tool (smolagents) | Pré-análise com regex/keywords |
| `OCRQualityTool` | Tool (smolagents) | Avalia qualidade do OCR |

### Fluxo Recomendado:

```
1. OCRQualityTool          → Avalia se texto está legível
       ↓
2. HeuristicAnalyzerTool   → Pré-identifica padrões (CPF, CNPJ, keywords)
       ↓
3. DocumentSegmenterTool   → Classifica cada segmento via LLM
       ↓
4. Validação cruzada       → Compara heurística vs LLM
       ↓
5. Extração de segmentos críticos → Para próximos agentes
```

### Próximos Passos:

- **SubsidyMatcherAgent**: Recebe segmentos `solicitacao_quebra_sigilo` e extrai subsídios
- **PartyExtractorAgent**: Recebe segmentos `informacoes_investigado` e extrai pessoas
- **DateExtractorAgent**: Extrai períodos dos segmentos críticos